In [2]:
!pip install numpy
!pip install wikipedia
!pip install lxml
!pip install geocoder
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes

     |████████████████████████████████| 102kB 26.3MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 5.8MB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 102kB 1.2MB/s ta 0:00:01
     |████████████████████████████████| 829kB 3.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 18.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/74/bc/50/ae030267a192919b289f84661cfeb5573cf383841c543e8696
Successfully built future
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7

In [3]:
import lxml
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm 
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import wikipedia as wp

## MIAMI NEIGHBORHOODS ( NAMES AND COORDINATES) 

Extracting list from wikipedia using wp method

In [4]:
html = wp.page("List_of_neighborhoods_in_Miami").html().encode("UTF-8")
df1 = pd.read_html(html)[0]
#df1.to_csv('beautifulsoup_pandas.csv',header=True,index=False)
print (df1.head())

                    Neighborhood      Demonym Population2010 Population/KmÂ²  \
0                     Allapattah          NaN          54289            4401   
1  Arts & Entertainment District          NaN          11033            7948   
2                       Brickell  Brickellite          31759           14541   
3                    Buena Vista          NaN           9058            3540   
4                  Coconut Grove      Grovite          20076            3091   

                                   Sub-neighborhoods    Coordinates  
0                                                NaN  25.815-80.224  
1                                                NaN  25.799-80.190  
2                                      West Brickell  25.758-80.193  
3  Buena Vista East Historic District and Design ...  25.813-80.192  
4  Center Grove, Northeast Coconut Grove, Southwe...  25.712-80.257  


In [5]:
df2 = df1.drop(['Demonym','Population2010','Population/KmÂ²','Sub-neighborhoods'], axis=1)
df2.head()

,Neighborhood,Coordinates
0,Allapattah,25.815-80.224
1,Arts & Entertainment District,25.799-80.190
2,Brickell,25.758-80.193
3,Buena Vista,25.813-80.192
4,Coconut Grove,25.712-80.257


Splitting coordinates in two columns. Had to be aware that in this case longitudes are all negative values. It is not a hyphen.

In [6]:
df3=df2.join(df2['Coordinates'].str.split('-', 1, expand = True).rename(columns={0:'Latitude', 1:'Longitude'}))
df4=df3.drop(['Coordinates'], axis=1)

In [7]:
df5 = df4.dropna()
df5.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,80.224
1,Arts & Entertainment District,25.799,80.190
2,Brickell,25.758,80.193
3,Buena Vista,25.813,80.192
4,Coconut Grove,25.712,80.257


Putting the negative values back. 

In [8]:
df6 = df5.astype({'Latitude': float, 'Longitude': float})
df6['Longitude'] *= -1
neighborhoods=df6
df6.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257


Plotting Miami map and its Neighborhoods

In [9]:
address = 'Miami'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Miami are 25.7742658, -80.1936589.


In [10]:
map_miami = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, Neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_miami)  
map_miami

Using Foursquare API of Venues categories in the city of Miami.

In [11]:
CLIENT_ID = 'KVJKJIDIIU3IJNCHOORWH5FRNDM5BE51XRTDC1G2DOZYOBF0' # your Foursquare ID
CLIENT_SECRET = '4MQHADELHR1GE2PWPKAGNZTLOYYROFE55FPOM0LVGCCNZP0J' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: KVJKJIDIIU3IJNCHOORWH5FRNDM5BE51XRTDC1G2DOZYOBF0
CLIENT_SECRET:4MQHADELHR1GE2PWPKAGNZTLOYYROFE55FPOM0LVGCCNZP0J


In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0] ['name']

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

List of neighborhoods

In [20]:
miami_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Midtown
Overtown
Park West
The Roads
Upper Eastside
Venetian Islands
Virginia Key
West Flagler
Wynwood


In [21]:
print(miami_venues.shape)
miami_venues.size

(549, 7)


3843

Number of different Venues in each neighborhood.
Total of 155 Unique category.

In [22]:
miami_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allapattah,4,4,4,4,4,4
Arts & Entertainment District,20,20,20,20,20,20
Brickell,57,57,57,57,57,57
Buena Vista,45,45,45,45,45,45
Coconut Grove,3,3,3,3,3,3
Coral Way,11,11,11,11,11,11
Design District,44,44,44,44,44,44
Downtown,66,66,66,66,66,66
Edgewater,44,44,44,44,44,44


In [23]:
print('There are {} uniques categories.'.format(len(miami_venues['Venue Category'].unique())))

There are 150 uniques categories.


Finding categories that are related with arts.Getting an array of all unique categories. 

In [24]:
cats=miami_venues['Venue Category'].unique()
cats

array(['Lounge', 'American Restaurant', 'Department Store',
       'Convenience Store', 'Cupcake Shop', 'Wine Shop', 'Spa',
       'Restaurant', 'Paper / Office Supplies Store', 'Sandwich Place',
       'Gym', 'Park', 'Ice Cream Shop', 'Pizza Place', 'Cuban Restaurant',
       'Tapas Restaurant', 'Salon / Barbershop', 'Smoothie Shop',
       'Coffee Shop', 'Beach', 'Food Truck', 'Hotel',
       'Argentinian Restaurant', 'Steakhouse', 'Japanese Restaurant',
       'Spanish Restaurant', 'Italian Restaurant', 'Dog Run', 'Café',
       'French Restaurant', 'Athletics & Sports',
       'Latin American Restaurant', 'Pharmacy', 'Scenic Lookout',
       'Gym / Fitness Center', 'Seafood Restaurant',
       'Mediterranean Restaurant', 'Nightclub', 'Burger Joint',
       'Juice Bar', 'Venezuelan Restaurant', 'Salad Place', 'Bank',
       'New American Restaurant', 'Mexican Restaurant', 'Bar',
       'Playground', 'Liquor Store',
       'Residential Building (Apartment / Condo)', 'Harbor / Marina'


Neighborhoods that are related with arts.

In [25]:
subs = 'Art'
res = list (filter(lambda x : subs in x, cats))
res

['Art Museum',
 'Art Gallery',
 'Martial Arts Dojo',
 'Performing Arts Venue',
 'Public Art']

# Neighboorhood with specific Art Categoty

Filtering the best Art category for our type of business.
Understanting the neighborhoods where we can find those category.

In [26]:
arts = miami_venues.loc[miami_venues['Venue Category'].isin(['Art Gallery','Public Art'])]
arts

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
112,Buena Vista,25.813,-80.192,Wyndwood,25.810577,-80.189607,Art Gallery
120,Buena Vista,25.813,-80.192,Locust Projects,25.812401,-80.195662,Art Gallery
175,Design District,25.813,-80.193,Locust Projects,25.812401,-80.195662,Art Gallery
177,Design District,25.813,-80.193,Wyndwood,25.810577,-80.189607,Art Gallery
263,Edgewater,25.802,-80.190,Art Wynwood,25.804871,-80.193170,Art Gallery
264,Edgewater,25.802,-80.190,Gary Nader Gallery,25.802323,-80.193634,Art Gallery
358,Midtown,25.807,-80.193,Art Wynwood,25.804871,-80.193170,Art Gallery
389,Midtown,25.807,-80.193,Spectrum Art Fair,25.804979,-80.193747,Art Gallery
421,Park West,25.785,-80.193,CIFO - Cisneros Fontanals Art Foundation,25.784011,-80.194211,Art Gallery
473,Wynwood,25.804,-80.199,Rubell Family Collection,25.803904,-80.196903,Art Gallery


# Conclusion
# Option 1
Now we know that the neighborhood that concentrates our two categories of interested is Wynwood.
We also know the other neighborhoods are very simmilar in number of art venues. We can open a store
in this neighborhood.

In [27]:
arts.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Buena Vista,2,2,2,2,2,2
Design District,2,2,2,2,2,2
Edgewater,2,2,2,2,2,2
Midtown,2,2,2,2,2,2
Park West,1,1,1,1,1,1
Wynwood,13,13,13,13,13,13


# Option 2
If retail space prices are too high in Wynwood, we can determine the middle points between
these neighborhoods and Venues. Would also be great to open a Store in the center of our interest.

In [28]:
arts.groupby('Neighborhood').mean()

,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
Neighborhood,,,,
Buena Vista,25.813,-80.192,25.811489,-80.192634
Design District,25.813,-80.193,25.811489,-80.192634
Edgewater,25.802,-80.190,25.803597,-80.193402
Midtown,25.807,-80.193,25.804925,-80.193459
Park West,25.785,-80.193,25.784011,-80.194211
Wynwood,25.804,-80.199,25.801089,-80.198889
